In [ ]:
!pip install optuna


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.0 MB/s eta 0:00:00


In [7]:
# Import necessary libraries
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, Subset
import torch.nn as nn
from torch.optim import SGD
import optuna
import numpy as np
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# Loading MNIST dataset from torchvision
train_set = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)

#Load the test set (Use the same thing as train_set)
test_set = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_set, batch_size=64, shuffle=False)


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 70699777.65it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 60416804.90it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

100%|██████████| 1648877/1648877 [00:00<00:00, 25961722.74it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 7580791.39it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
import optuna
import numpy as np
# Define the model architecture
class NN_classification(nn.Module):
    def __init__(self, n_kernels1, n_kernels2, kernel_size1, kernel_size2, padding1, padding2, stride1, stride2):
        super(NN_classification, self).__init__()
        self.conv1 = nn.Conv2d(1, n_kernels1, kernel_size=kernel_size1, padding=padding1, stride=stride1)
        self.conv2 = nn.Conv2d(n_kernels1, n_kernels2, kernel_size=kernel_size2, padding=padding2, stride=stride2)
        self.fc1 = nn.Linear(self._get_conv_output((1, 28, 28)), 128)  # Adjust based on the actual output size
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST

    def _get_conv_output(self, shape):
        # Create a dummy tensor with the given shape and pass it through the network
        with torch.no_grad():
            x = torch.zeros(1, *shape)
            x = F.relu(self.conv1(x))
            x = F.max_pool2d(x, 2)
            x = F.relu(self.conv2(x))
            x = F.max_pool2d(x, 2)
            return int(np.prod(x.size()))  # Get the flattened size

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    n_kernels1 = trial.suggest_int('n_kernels1', 16, 64)
    n_kernels2 = trial.suggest_int('n_kernels2', 32, 128)
    kernel_size1 = trial.suggest_int('kernel_size1', 3, 7)
    kernel_size2 = trial.suggest_int('kernel_size2', 3, 7)
    padding1 = trial.suggest_int('padding1', 0, 2)
    padding2 = trial.suggest_int('padding2', 0, 2)
    stride1 = trial.suggest_int('stride1', 1, 2)
    stride2 = trial.suggest_int('stride2', 1, 2)

    # Create the model
    model = NN_classification(n_kernels1, n_kernels2, kernel_size1, kernel_size2, padding1, padding2, stride1, stride2)

    # Define the optimizer and loss function
    optimizer = SGD(model.parameters(), lr=0.01)
    loss_fn = nn.CrossEntropyLoss()

    # Train the model on the entire training set
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            predictions = model(images)
            loss = loss_fn(predictions, labels)
            loss.backward()
            optimizer.step()
            running_train_loss += loss.item()

    # Evaluate on the test set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            predictions = model(images)
            _, predicted_labels = torch.max(predictions, 1)
            total += labels.size(0)
            correct += (predicted_labels == labels).sum().item()

    test_accuracy = correct / total
    return test_accuracy

# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)

[I 2024-07-31 16:19:44,131] A new study created in memory with name: no-name-02b953f8-716b-40a4-8641-3435d26ac4f5
[I 2024-07-31 16:53:46,712] Trial 0 finished with value: 0.9899 and parameters: {'n_kernels1': 63, 'n_kernels2': 71, 'kernel_size1': 6, 'kernel_size2': 5, 'padding1': 0, 'padding2': 1, 'stride1': 2, 'stride2': 2}. Best is trial 0 with value: 0.9899.


# after finding the best hyperparameters for our model , we are going to re-train our CNN model with these hyperparameters and then evaluate it on a test data.

In [21]:
import torch.nn as nn
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 63, kernel_size=6, stride=2, padding=0)
        self.conv2 = nn.Conv2d(63, 71, kernel_size=5, stride=2, padding=1)
        self.fc1 = nn.Linear(71 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = x.view(-1, 71 * 5 * 5)  # Corrected view size
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = NeuralNet()


In [23]:


from torch.optim import SGD

num_epochs = 10

optimizer = SGD(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    running_loss = 0.0

    for images, labels in train_loader:

        predictions = model(images)


        loss = loss_fn(predictions, labels)


        loss.backward()

        optimizer.step()


        optimizer.zero_grad()

        running_loss += loss.item()

    # Print the average loss for the epoch
    print(f"Epoch {epoch+1}, Loss {running_loss/len(train_loader)} ")

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())


Epoch 1, Loss 0.03894274022167316 
Epoch 2, Loss 0.034041424936576206 
Epoch 3, Loss 0.03149656269175342 
Epoch 4, Loss 0.029612942539187257 
Epoch 5, Loss 0.027831871736249122 
Epoch 6, Loss 0.026259300231286732 
Epoch 7, Loss 0.02492477313891775 
Epoch 8, Loss 0.023657638063557 
Epoch 9, Loss 0.022444046147764404 
Epoch 10, Loss 0.021262303916680346 
Model's state_dict:
conv1.weight 	 torch.Size([63, 1, 6, 6])
conv1.bias 	 torch.Size([63])
conv2.weight 	 torch.Size([71, 63, 5, 5])
conv2.bias 	 torch.Size([71])
fc1.weight 	 torch.Size([128, 1775])
fc1.bias 	 torch.Size([128])
fc2.weight 	 torch.Size([10, 128])
fc2.bias 	 torch.Size([10])


# **Evaluation on the test dataset**

In [24]:
model.eval()
test_loss = 0.0
for images, labels in test_loader:
    with torch.no_grad():
        predictions = model(images)
        loss = loss_fn(predictions, labels)
        test_loss += loss.item()

print(f"Loss: {test_loss/len(test_loader)}")


Loss: 0.03425671447144086


# **here we got a loss function smaller than the one we got when taking random hyperparamters values**